In [ ]:
%%time

import itertools
import os 
import sys
import re
import math
import random
import requests
import seaborn as sns
import matplotlib.pyplot as plt 
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.colors import ListedColormap
from matplotlib.ticker import PercentFormatter
import matplotlib.pylab as plt
import numpy as np
from numpy import asarray
from numpy import save
from numpy import load
import PIL.Image as Image
import pandas as pd
import tensorflow as tf
import keras
from tensorflow.keras.layers import Input
from keras.models import Sequential, Model
from keras.layers import BatchNormalization, Flatten, Input, Dense,Dropout,Activation,Flatten,Reshape,Lambda,Embedding,Conv2D,MaxPooling2D, UpSampling2D   #卷积层，池化层
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.optimizers import Adam
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import tensorflow.python.platform.build_info as build 
import tensorflow_hub as hub 
import tensorflow_datasets as tfds 
from tensorflow.python.client import device_lib

def getT0_all():
  t0_filename = "TEMPERATURE_at_frame240.txt"  
  time_stamp_raw = pd.read_csv(t0_filename, sep = '\t', header = None, 
                                skiprows=[i for i in range(0,4)], on_bad_lines='skip')
  timestamp = [float(s) for s in re.findall(r':\s(.*)', time_stamp_raw.iat[2, 1])]
  timestamp = timestamp[0]    # float number for timestamp
    
  temp_raw = pd.read_csv(t0_filename, header = None, sep = '\t', 
                        skiprows=[i for i in range(0,9)]).iloc[[i for i in range(0, 17574-9)], [0, 1]]
  temp_raw.columns = ['node_label', 'temperature']
  temp = pd.merge(temp_raw, node_info, on = 'node_label')
  temp['temperature'] = temp['temperature'] - 273.15

  initial_time = 0
  x_dim = (temp['node_x'].unique()).size;
  y_dim = (temp['node_y'].unique()).size;
  z_dim = (temp['node_z'].unique()).size;

  temp['time'] = (timestamp - initial_time)/60    #time is in minutes
  temp['scaled_x'] = ( temp['node_x'] / max(temp['node_x']) ) * x_dim
  temp['scaled_y'] = ( (temp['node_y']) / (max(temp['node_y'])) ) * y_dim
  temp['scaled_z'] = ( (temp['node_z']) / (max(temp['node_z'])) ) * z_dim

  # cast original coordinates values into integers
  temp['X'] = temp.scaled_x.astype('int');   
  temp['Y'] = temp.scaled_y.astype('int');
  temp['Z'] = temp.scaled_z.astype('int');
  t0_info_all.append(temp)       ##### append current frame to info DF #####
  
  # cast temperature data into images
  img_x = max(temp['X']) +1      
  img_y = max(temp['Y']) +1      
  img_z = max(temp['Z']) +1      
  get_image = np.zeros(shape = (img_y, img_x, img_z))

  for i in range(len(temp)):
    x_coordinate = temp.loc[i, "X"]
    y_coordinate = temp.loc[i, "Y"]
    z_coordinate = temp.loc[i, "Z"]
    get_image[y_coordinate][x_coordinate][z_coordinate] = temp.loc[i, "temperature"]
  t0_img_all.append(get_image)   ##### append current frame to img DF #####

def getInput(folder_name):   
  input =[]
  split_str = folder_name.split("_")
  input.append(split_str[2])
  input.append(split_str[3])
  input.append(split_str[4])
  input.append(split_str[5])
  return input

def getImgChannel(channel, dataset):
  output_img = []
  length = len(dataset)
  for i in range(length):
    output_img.append(dataset[i, :, :, channel])
  return np.array(output_img)

def prep_Input_parameters(inputs):
  dic0 = {'1': 41.0,  '2': 43.0,   '3': 44.0,  '4': 45.0,   '5': 46.0}
  dic1 = {'1': 79.0,  '2': 80.0,  '3': 81.0,  '4': 82.0,  '5': 83.0}
  dic2 = {'1': 79.0, '2': 80.0, '3': 81.0, '4': 82.0, '5': 83.0}
  dic3 = {'1': 20.0,  '2': 21.0,  '3': 22.0,  '4': 23.0,  '5': 25.0}
  new_inputs_all = []
  for i in range(len(inputs)):
    this_input = []
    for j in range(len(inputs[i])):
      the_value = (inputs[i])[j]
      if j == 0:
        this_input.append(dic0[the_value])
      elif j == 1:
        this_input.append(dic1[the_value])
      elif j == 2:
        this_input.append(dic2[the_value])
      else:
        this_input.append(dic3[the_value])
        new_inputs_all.append(this_input)
  return new_inputs_all

def patch_image(dataset):
  for img in range(len(dataset)):
    for col in range(102):
      (dataset[img])[28, col] = ((dataset[img])[27, col] + (dataset[img])[29, col]) / 2
    for row in range(30):
      (dataset[img])[row, 100] = ((dataset[img])[row, 99] + (dataset[img])[row, 101]) / 2
    (dataset[img])[28, 100] = ((dataset[img])[27, 99] + (dataset[img])[27, 101] + (dataset[img])[29, 99] + (dataset[img])[29, 101]) / 4
    for row1 in range(27, 30):
        for col1 in range(99, 102):
            (dataset[img])[row1, col1] = ((dataset[img])[row1, col1-1] + (dataset[img])[row1-1, col1]) / 2
################################################################################
%cd C:\A4-21 3D model temperature control data set

nodes_amt = 17574      # total nodes 
node_info = pd.read_csv('Mesh_3D_2m_0.56m_0.06m.txt', delim_whitespace = True, header = None)
node_info.columns = ['node_label', 'node_x', 'node_y', 'node_z']
node_info = node_info.astype(str).astype(float)

root_dir = os.getcwd()
folders = os.listdir(root_dir)

input_parameters_all = []     # raw inputs for embeddings
t0_img_all = []               # starting frame as output y for training
t0_info_all = []

num_parameters = 4            # not used yet
temperature_info_all = []     # hold all temperature info of all datasets
temperature_img_all = []      # hold all temperature imgs of all datasets

for name in folders:
  if (len(name)) == 20:
    input_parameters_all.append(getInput(name))
    cwd = "C:/WindSTAR/A4-21 3D model temperature control data set/{}".format(name)  
    os.chdir(cwd)             # change directory 
    getT0_all()
    
# convert to Numpy array
t0_img_all = np.array(t0_img_all)
input_parameters_all = np.array(input_parameters_all)
inputs_all = np.array(prep_Input_parameters(input_parameters_all))  # Convert parameters to real values

All_img = []
All_labels = []
total_perm = 625

for i in range(6):
    which_layer = i
    if(i == 5): which_layer = i + 1
    current_layer_img = getImgChannel(which_layer, t0_img_all)
    new_input = np.copy(inputs_all)
    current_layer = np.full((total_perm, 1), i + 1)
    new_input = np.hstack((new_input, current_layer))
    for j in range(total_perm): 
        All_img.append(current_layer_img[j])
        All_labels.append(new_input[j])
        
All_img = np.array(All_img)        #(3750, 30, 102)
patch_image(All_img)
All_img_norm = All_img / np.max(All_img)
All_labels = np.array(All_labels)  #(3750, 30, 102)

# normalize each column in the array labels
max_val_labels = []
min_val_labels = []
for col in range(5):
    max_val = np.max(All_labels[:,col])
    min_val = np.min(All_labels[:,col])
    max_val_labels.append(max_val)
    min_val_labels.append(min_val)
    All_labels[:,col] = (All_labels[:,col] - min_val) / (max_val - min_val)
    
max_val_labels = np.array(max_val_labels)
min_val_labels = np.array(min_val_labels)

X_train_all, X_test, y_train_all, y_test = train_test_split(All_labels, All_img_norm, 
                                                    test_size=0.2, random_state=42)
y_test = y_test.reshape(y_test.shape[0], y_test.shape[1], y_test.shape[2], 1)

X_train, X_val, y_train, y_val = train_test_split(X_train_all, y_train_all, 
                                                    test_size=0.2, random_state=42)
y_train = y_train.reshape(y_train.shape[0], y_train.shape[1], y_train.shape[2], 1)
y_val = y_val.reshape(y_val.shape[0], y_val.shape[1], y_val.shape[2], 1)
# X_train.shape = (2400, 5)
# X_test.shape = (750, 5)
# X_val.shape = (600, 5)
# y_train.shape = (2400, 30, 102, 1)
# y_test.shape = (750, 30, 102, 1)
# y_val.shape = (600, 30, 102, 1)

In [ ]:
%%time

par_net = []
input_par = Input(shape = (5,), name = 'Input') 
x = Reshape((1, 1, 5), name = 'reshape1')(input_par)
x = UpSampling2D((2, 2), name = 'up1')(x)
x = Conv2D(128, (3, 3), activation = 'relu', padding = 'same', name = 'conv1')(x)
x = UpSampling2D((2, 2), name = 'up2')(x)
x = Conv2D(64, (3, 3), activation = 'relu', padding = 'same', name = 'conv2')(x)
x = UpSampling2D((3, 3), name = 'up3')(x)
x = Conv2D(32, (3, 3), activation = 'relu', name = 'conv3')(x)
x = UpSampling2D((1, 2), name = 'up4')(x)
x = Conv2D(16, (3, 3), activation = 'relu', name = 'conv4')(x)
x = UpSampling2D((2, 3), name = 'up5')(x)
x = Conv2D(16, (2, 3), activation = 'relu', name = 'conv5')(x)
x = UpSampling2D((2, 2), name = 'up6')(x)
par_net = Conv2D(1, (1, 3), activation = 'relu', name = 'conv6')(x)
par_net = Model(input_par, par_net)
par_net.summary()

batch_size = 64
lr = 0.001
epochs = 1500
callbacks = [EarlyStopping(monitor = 'val_loss', patience = 100, verbose = 2),
             ModelCheckpoint('Par-net.h5', monitor = 'val_loss', save_best_only = True, verbose=0)]
keras.backend.set_epsilon(1)
par_net.compile(loss = 'mae', optimizer = keras.optimizers.Adam(learning_rate = lr),  
                 metrics=['mape'])
history = par_net.fit(X_train, y_train, batch_size = batch_size, epochs = epochs, workers=8,
                      use_multiprocessing = 1, callbacks = callbacks, 
                      validation_data = (X_val, y_val))

plt.plot(history.history['mape'], label = 'Training data')
plt.plot(history.history['val_mape'], label ='Validation data')
plt.ylabel('MAPE (%)')
plt.xlabel('No. epoch')
plt.ylim(0, 25)
plt.legend(loc = 'upper right', borderaxespad = 0, frameon = False)
font = {'family' : 'normal', 'weight' : 'normal', 'size'   : 14}
plt.rc('font', **font)
plt.show()

In [ ]:
par_net_testing = par_net.evaluate(X_test, y_test, verbose = 2)
par_net_testing

In [ ]:
%%time
#### Load the trained par-net model and compare predictions with original data

def plot_images_comparison(original, predicted, mape, process_par_str):
  fig, (ax1, ax2, ax3) = plt.subplots(nrows = 3, ncols=1, figsize=(8, 9), sharex = True)
  plt.tight_layout(pad = 1)
  ax1.set_title('Processing parameters: ' + "\n" + 'T1 = ' + str(process_par_str[0]) + ' °C, ' + 
                'T2 = ' + str(process_par_str[1]) + ' °C, ' + 
                'T3 = ' + str(process_par_str[2]) + ' °C, ' + 
                'T4 = ' + str(process_par_str[3]) + ' °C, ' + 
                'layer ' + str(process_par_str[4]) + 
                "\n" + "\n" + 'True temperature (°C)', pad=11, fontsize = font_size)
  im1 = ax1.imshow(original, cmap='jet', aspect='auto')

  ax2.set_title('Predicted temperature (°C)', pad=11, fontsize = font_size)
  im2 = ax2.imshow(predicted, cmap='jet', aspect='auto')

  avg_MAPE = np.round_(np.sum(mape) / (mape.shape[0] * mape.shape[1]), decimals  = 2)
  ax3.set_title('MAPE (%) mapping with average ' + str(round(avg_MAPE, 1)) + ' %', 
                pad=11, fontsize = font_size)
  im3 = ax3.imshow(mape, cmap='jet', aspect='auto')
  
  #plt.setp(ax[0], xlabel='Length of composite (pixels)')
  plt.setp(ax3, xlabel='Length of composite (pixels)')
  plt.setp(ax2, ylabel='Width of composite (pixels)')

  # add space for colour bar
  fig.subplots_adjust(right = 0.86)
  cbar_ax1 = fig.add_axes([0.89, 0.48, 0.01, 0.3])
  cbar1_ticks = np.around(np.linspace(np.around(np.min(predicted), decimals = 1), 
                            np.around(np.max(predicted), decimals = 1), 6, endpoint=True), decimals = 1)
  cbar1 = fig.colorbar(im2, cax = cbar_ax1)
  cbar1.set_ticks(cbar1_ticks)
  cbar1.set_ticklabels((np.around(cbar1_ticks, decimals = 0)).astype(int))
  ax1.text(1.04, 0.55, '(°C)', transform=ax1.transAxes, fontsize = font_size, verticalalignment='top')

  cbar_ax2 = fig.add_axes([0.89, 0.1, 0.01, 0.15])
  cbar2_ticks = np.around(np.linspace(np.around(np.min(mape), decimals = 1), 
                            np.around(np.max(mape), decimals = 1), 4, endpoint=True), decimals = 1)
  cbar2 = fig.colorbar(im3, cax = cbar_ax2)
  cbar2.set_ticks(cbar2_ticks)
  cbar2.set_ticklabels(cbar2_ticks)
  ax2.text(1.04, -0.35, '(%)', transform=ax2.transAxes, fontsize = font_size, verticalalignment='top')

  font = {'family': 'Times New Roman', 'weight': 'normal', 'size': font_size}
  plt.rc('font', **font)

def calculate_MAPE_mapping(ori_data, pred_data):
  mape_mapping_all = []
  row = ori_data[0].shape[0]
  col = ori_data[0].shape[1]
  for frame in range(len(ori_data)):
    current_ori = ori_data[frame]
    current_pred = pred_data[frame]
    current_mape = np.zeros(shape = (row, col))
    for y in range(row):
      for x in range(col):
        ori_val = current_ori[y][x]
        pred_val = current_pred[y][x]
        current_mape[y][x] = np.absolute(ori_val - pred_val) / (ori_val + 0.0000000001) * 100
    mape_mapping_all.append(current_mape)
  return np.array(mape_mapping_all)

### main 
five_par_all_X = np.vstack([X_train, X_val, X_test])
five_par_all_Y = np.vstack([y_train, y_val, y_test])
Pred_array = par_net.predict(five_par_all_X, batch_size = 128)    
Prediction_norm = Pred_array.reshape(Pred_array.shape[0], Pred_array.shape[1], Pred_array.shape[2])
Prediction = Prediction_norm * np.max(All_img)
original_img = five_par_all_Y * np.max(All_img)
mape_mapping = calculate_MAPE_mapping(original_img, Prediction)

##### Pick a input processing parameters to compare results 

whichStep = 2001
factor_label = [5, 4, 4, 5, 5]
base_label = [41, 79, 79, 20, 1]
font_size = 24

process_par_num = (five_par_all_X[whichStep] * factor_label + base_label).astype(int)
mape_at_frame = mape_mapping[whichStep]
the_pred = Prediction[whichStep]
ori = original_img[whichStep]
plot_images_comparison(ori, the_pred, mape_at_frame, process_par_num)
avg_MAPE_test = []
for i in range(len(mape_mapping)):
    mape = mape_mapping[i]
    avg_MAPE = np.round_(np.sum(mape) / (mape.shape[0] * mape.shape[1]), decimals  = 2)
    avg_MAPE_test.append(avg_MAPE)
avg_MAPE_test = np.array(avg_MAPE_test)

In [ ]:
counts, edges, bars = plt.hist(avg_MAPE_test, 
                               weights = np.ones(len(avg_MAPE_test)) / len(avg_MAPE_test),
                               bins=[0, 5, 10, 30, 40, 50, 100])
plt.text(-4.1, 1.05, '0.972', fontsize = 14, verticalalignment='top')
plt.xlabel("MAPE of the prediction (%)")
plt.ylabel("Relative frequency")
plt.ylim([0, 1.06])
plt.xticks(np.arange(0, 110, 10))
plt.yticks(np.arange(0, 1.2, 0.2))
font = {'family': 'normal', 'weight': 'normal', 'size': 14}
plt.rc('font', **font)
plt.show()